# 問題1　機械翻訳の実行とコードリーディング

下記圧縮ファイルをダウンロードし、回答して、このsprint24.ipynbと同じ階層においてください。

http://www.manythings.org/anki/fra-eng.zip


次に、下記githubにアクセスし、プログラムをコピペしてきます。

https://github.com/rstudio/keras/blob/master/vignettes/examples/lstm_seq2seq.py

下記は、コピペしてきたプログラムを分かりやすく分解したものになります。

【アルゴリズム参考】

https://blog.octopt.com/sequence-to-sequence/

【プログラム参考】

https://omathin.com/seq2seq-study/

https://qiita.com/FukuharaYohei/items/27cd247342a0f7006511

##### ダイバー
以下のサンプルコードは、短い英語からフランス語への変換を行うものです。これを動かしてください。


keras/lstm_seq2seq.py at master · rstudio/keras


その上でこのサンプルコードの各部分がどういった役割かを読み取り、まとめてください。以下のようにどこからどこの行が何をしているかを記述してください。


（例）


51から55行目 : ライブラリのimport
57から62行目 : ハイパーパラメータの設定

《文字単位のトークン化》


この実装ではテキストのベクトル化の際に、単語ではなく文字ごとを1つのトークンとして扱っています。


scikit-learnでBoWを計算するCountVectorizerの場合では、デフォルトの引数はanalyzer=’word’で単語を扱いますが、charやchar_wbとすることで文字を扱えるようになります。


charとchar_wbの2種類の方法があり、char_wbを指定した場合、n_gramが単語内からのみ作成されます。逆にcharは単語の区切りが関係なくn_gramが作成されます。This movie is very good.というテキストを3-gramでカウントする時、charではs mやe iといった単語をまたぐ数え方もしますが、char_wbではこれらを見ません。


sklearn.feature_extraction.text.CountVectorizer — scikit-learn 0.21.3 documentation

In [96]:
%pwd

'/content/drive/My Drive/Colab Notebooks/Sprint24 Seq2Seq/pytorch-tutorial/tutorials/03-advanced/image_captioning/pytorch-tutorial/tutorials/03-advanced/image_captioning/pytorch-tutorial/tutorials/03-advanced/image_captioning/pytorch-tutorial/tutorials/03-advanced/image_captioning/pytorch-tutorial/tutorials/03-advanced/image_captioning/pytorch-tutorial/tutorials/03-advanced/image_captioning/pytorch-tutorial/tutorials/03-advanced/image_captioning'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [95]:
%ls

build_vocab.py  model.py      README.md         sample.py
data_loader.py  png/          requirements.txt  train.py
download.sh     __pycache__/  resize.py


In [ ]:
%cd Sprint24 Seq2Seq

[Errno 2] No such file or directory: 'Sprint24 Seq2Seq'
/content/drive/My Drive/Colab Notebooks/Sprint24 Seq2Seq/pytorch-tutorial/tutorials/03-advanced/image_captioning/pytorch-tutorial/tutorials/03-advanced/image_captioning/pytorch-tutorial/tutorials/03-advanced/image_captioning/pytorch-tutorial/tutorials/03-advanced/image_captioning/pytorch-tutorial/tutorials/03-advanced/image_captioning/pytorch-tutorial/tutorials/03-advanced/image_captioning/pytorch-tutorial/tutorials/03-advanced/image_captioning


In [ ]:
# ライブラリのimport
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [ ]:
# 必要な変数の定義
## 学習関係
batch_size = 1  #64 バッチサイズ.
epochs = 1  #10 学習回数
latent_dim = 256  # 
num_samples = 10000  # サンプル数

## ファイル関係
data_path = 'fra-eng/fra.txt' # データパス

## データ関係
input_texts = [] # 入力テキスト（説明変数）
target_texts = [] # 翻訳テキスト（目的変数）
input_characters = set() # 一意な入力文字
target_characters = set() # 一意な翻訳文字

In [ ]:
# データの読み込み
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

# 取得した行でループ
for line in lines[: min(num_samples, len(lines) - 1)]:
    # タブで区切られている説明変数と目的変数を分離
    input_text, target_text, _ = line.split('\t')
    # タブと改行で挟む
    target_text = '\t' + target_text + '\n'
    # 説明変数と目的変数をそれぞれ追加
    input_texts.append(input_text)
    target_texts.append(target_text)
    # 一意な文字それぞれ保存
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

FileNotFoundError: ignored

In [ ]:
# 学習推定に必要な変数の定義
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

In [ ]:
# 一意な文字の辞書化
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

# 入力サンプル数　×　最大文字数　×　一意な文字数（エンコーダ：特徴抽出）
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
# 入力サンプル数　×　最大文字数　×　一意な文字数（デコーダ：翻訳）
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
# 入力サンプル数　×　最大文字数　×　一意な文字数（デコーダ：翻訳）
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [ ]:
# 初期化していたencoder_input_data/decoder_input_data/decoder_target_dataの該当箇所に1を代入（one-hot）
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [ ]:
# 入力データ数は未定・入力特徴量数はnum_encoder_tokens（一意な文字数）
encoder_inputs = Input(shape=(None, num_encoder_tokens))
# LSTM（エンコーダ）
encoder = LSTM(latent_dim, return_state=True)
# 中間状態取得
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [ ]:
# 入力データ数は未定・入力特徴量数はnum_decoder_tokens（一意な文字数）
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# LSTM（デコーダ）
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
# 通常層に入れる
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
# モデル結合
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# コンパイル
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# 学習と保存
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2
)

model.save('s2s.h5')

In [ ]:
# エンコーダ
encoder_model = Model(encoder_inputs, encoder_states)

# デコーダ
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, 
    initial_state=decoder_states_inputs
)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

In [ ]:
# 予測から文字列に起こすための辞書
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
    # 入力ベクトルをエンコーダへ
    states_value = encoder_model.predict(input_seq)

    # 0で初期化
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # 先頭にタブ
    target_seq[0, 0, target_token_index['\t']] = 1.

    # 条件を満たすまでループ
    stop_condition = False
    # 最終的な文章
    decoded_sentence = ''
    while not stop_condition:
        # 予測
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # 予測値を確定させ、数値から文字列に復元させる
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # 最大文字数に達するか改行文字
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # デコーダへの入力をupdate
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        states_value = [h, c]

    return decoded_sentence

In [ ]:
# 100文章分ループ
for seq_index in range(10): #100
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

# 問題2　イメージキャプショニングの学習済みモデルの実行

他の活用例としてイメージキャプショニングがあります。画像に対する説明の文章を推定するタスクです。これは画像を入力し、系列データを出力する Image to Sequence の手法によって行えます。


pytorch-tutorial/tutorials/03-advanced/image_captioning at master · yunjey/pytorch-tutorial


イメージキャプショニングは学習に多くの時間がかかるため、ここでは学習済みの重みが公開されている実装を動かすことにします。Kerasには平易に扱える実装が公開されていないため、今回はPyTorchによる実装を扱います。



上記実装において 5. Test the model の項目を実行してください。また、自身で用意した画像に対しても文章を生成してください。これらに対してどういった文章が出力されたかを記録して提出してください。


データセットからの学習は行わず、学習済みの重みをダウンロードして利用します。


注意点として、デフォルトで設定されている重みのファイル名と、ダウンロードできる重みのファイル名は異なっています。ここは書き換える必要があります。

In [ ]:
# レポジトリからチュートリアルプログラム取得
!git clone https://github.com/yunjey/pytorch-tutorial.git

In [ ]:
# 実行したいpyファイルのある場所に移動
%cd pytorch-tutorial/tutorials/03-advanced/image_captioning/

In [ ]:
# 必要モジュールのインストール
!pip install -r requirements.txt
!pip install torchvision 
!pip install pycocotools

In [ ]:
# ライブラリimport
import torch
import torchvision
%matplotlib inline

In [ ]:
# 試しに出力
from IPython.display import Image,display_png
import pickle


with open('serialized.pkl', 'rb') as f:
    data = pickle.load(f)display_png(Image('png/example.png'))


サンプルプログラム実行のためには、下記のURLの記載に従って、学習済みモデルをダウンロードしてくる必要がある

https://github.com/yunjey/pytorch-tutorial/tree/master/tutorials/03-advanced/image_captioning#pretrained-model

そしてそれぞれのファイルを、下記のようにディレクトリに配置する。

pytorch-tutorial/tutorials/03-advanced/image_captioning/data/vocab.pkl

pytorch-tutorial/tutorials/03-advanced/image_captioning/models/decoder-5-3000.pkl

pytorch-tutorial/tutorials/03-advanced/image_captioning/models/encoder-5-3000.pkl

In [94]:
# サンプルプログラム実行
!python sample.py --image="png/example.png"

Traceback (most recent call last):
  File "sample.py", line 81, in <module>
    main(args)
  File "sample.py", line 33, in main
    with open(args.vocab_path, 'rb') as f:
FileNotFoundError: [Errno 2] No such file or directory: 'data/vocab.pkl'


# 問題3　Kerasで動かしたい場合はどうするかを調査

PyTorchによる実装を動かしましたが、何らかの理由からKerasで動かしたい状況が考えられます。どういった手順を踏むことになるか調査し、できるだけ詳しく説明してください。


特に今回はPyTorchのための学習済みの重みをKerasで使えるようにしたいので、その点については必ず触れてください。

下記のようなOSSのツールを利用する

https://github.com/gmalivenko/pytorch2keras

https://github.com/Microsoft/MMdnn